# Задание 5.2
1. Начальная популяция:
    - 1.2 жадный выбор, начиная со случайного груза
2. Отбор особей для скрещивания:
    - 2.2 выбрать только 20% самых приспособленных особей
3. Скрещивание (кроссинговер) между выбранными особями. Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка:
    - 3.2 однородный (каждый бит от случайно выбранного родителя)
4. Мутация:
    - 4.2 случайное изменение 3х битов у 5% особей
5. Формирование новой популяции (кол-во особей - константа):
    - 5.3 замена своих родителей


In [2]:
import random
import math

In [3]:
FNAME = "17.txt"
NUMBER_OF_INDIVIDUALS = 200
GENERATIONS = 500

In [4]:
with open(FNAME) as f:
    content = f.readlines()
    
content = [x.strip() for x in content]
capacities = [float(n) for n in content.pop(0).split()]
print('weight: ', capacities[0], "\nvolume: ", capacities[1])

maxWeight = capacities[0]
maxVolume = capacities[1]

data = []
for x in content:
    data.append([float(n) for n in x.split()])
    
data = sorted(data, key=lambda x: x[2], reverse=True)
data

weight:  13000.0 
volume:  12.0


[[1149.0, 1.2, 384.0],
 [602.0, 1.1, 374.0],
 [1278.0, 0.5, 351.0],
 [698.0, 1.2, 350.0],
 [1474.0, 0.7, 321.0],
 [740.0, 1.0, 317.0],
 [154.0, 0.7, 308.0],
 [1446.0, 0.5, 297.0],
 [174.0, 0.9, 297.0],
 [1192.0, 1.2, 289.0],
 [893.0, 0.9, 280.0],
 [864.0, 0.4, 244.0],
 [856.0, 0.6, 238.0],
 [945.0, 0.8, 236.0],
 [133.0, 0.5, 234.0],
 [579.0, 0.4, 229.0],
 [214.0, 0.6, 221.0],
 [1688.0, 0.4, 214.0],
 [1240.0, 1.1, 213.0],
 [950.0, 1.0, 212.0],
 [1045.0, 0.6, 211.0],
 [1112.0, 1.1, 173.0],
 [145.0, 0.9, 161.0],
 [853.0, 0.8, 145.0],
 [776.0, 0.9, 143.0],
 [1078.0, 0.9, 127.0],
 [1015.0, 0.4, 124.0],
 [579.0, 0.6, 121.0],
 [1192.0, 1.0, 120.0],
 [1571.0, 0.7, 115.0]]

In [5]:
def _starting_population():
    population = []
    for i in range(0, NUMBER_OF_INDIVIDUALS):
        population.append(_generate_individual());
    return population

def _generate_individual(): #генерируем заранее приспособленных особей (так как в задании непонятно)
    res = []
    quantity = 30
    start = random.randint(0, quantity - 1)

    bits_counter = 0
    bits_remaining = quantity - start

    weight = 0
    volume = 0
    value = 0
    
    for i in range (0, start):
        res.append(0)
    
    for i in range(start, quantity):
        
        if (weight+data[i][0] > maxWeight or volume+data[i][1] > maxVolume or bits_counter > bits_remaining): 
            break;
            
        res.append(1)    
        weight+=data[i][0]
        volume+=data[i][1]    
        value+=data[i][2]
        bits_counter += 1
            
    for i in range (bits_counter, bits_remaining):
        res.append(0)
    tmp = ''.join([str(x) for x in res])
    
    return { 'gen' : tmp, 'weight': weight, 'volume': round(volume,1), 'value': round(value,1) }

In [6]:
def _validate_individ(individ):
    weight = individ['weight']
    volume = individ['volume']
    if (weight > maxWeight or volume > maxVolume):
        return False;
    return True

def selection(population, s=20):
    selectioner = int(len(population) * s / 100) 
    population = sorted(population, key = lambda i: i['value'],reverse=True)
    
    selected = []
    not_selected = []
    selected_counter = 0
    
    for individ in population:

        if (not _validate_individ(individ)):
            not_selected.append(individ)
        elif(selected_counter < selectioner):
            selected.append(individ)
            selected_counter += 1
        else:
            not_selected.append(individ)
            
    return not_selected, selected
    

In [29]:
def _rand(start, stop, count):# Получаем неповторяющиеся случайные числа
        gamma = []
        for i in range(0, count):
            while True:
                item = random.randint(start, stop)
                if not gamma.count(item):
                    gamma.append(item)
                    yield item
                    break

def _crossingover(parent_1, parent_2):
    child_1 =''
    child_2 =''
    for i in range(0,30):
        rnd = random.getrandbits(1)
        if rnd == 0:
            child_1 += parent_1["gen"][i]
            child_2 += parent_2["gen"][i]
        if rnd == 1:
            child_1 += parent_2["gen"][i]
            child_2 += parent_1["gen"][i]
    return child_1, child_2;
                
def _get_child(gen):
    child_weight = 0
    child_volume = 0
    child_value = 0
    
    for i in range(0,len(gen)):
        if (int(gen[i]) == 1):
            child_weight += data[i][0]
            child_volume += data[i][1]
            child_value += data[i][2]
    return { 'gen' : gen, 'weight': child_weight, 'volume': round(child_volume,1), 'value': round(child_value,1) }

def _get_children(population):
    children = []
    not_used_parents = []
    priorities = list(_rand(0,len(population)-1,len(population)))
    for i in range(0,len(priorities),2):
        try:
            parent_1 = population[priorities[i]]
            parent_2 = population[priorities[i + 1]]
            gens = _crossingover(parent_1, parent_2)
            child_1_gen = gens[0]
            child_2_gen = gens[1]

            children.append(_get_child(child_1_gen))
            children.append(_get_child(child_2_gen))
        except: 
            not_used_parents.append(parent_1)
    return children, not_used_parents

def _individual_mutation(individ): #мутирует 3 случайных бита
    gen = list(map(int,list(individ['gen'])));
    random_bits = _rand(0, len(gen) - 1, 3)
    for b in random_bits:
        if (gen[b] == 0): gen[b] = 1;
        elif (gen[b] == 1): gen[b] = 0;
    new_gen = ''.join([str(x) for x in gen]);
    return _get_child(new_gen)
    

def _mutation(population):
    random_individuals = _rand(0, len(population) - 1, int(len(population) * 0.05))
    for individ in random_individuals:
        new_individ = _individual_mutation(population[individ])
        population[individ] = new_individ
    return population

def _check_population(population):
    last = population[len(population) - 1]["value"]
    first = population[0]["value"]
    percentasge = round(100 - (last/first)*100,2)
    return percentasge

def _print_population(population):
    for i in population:
        print(i)
        
def _find_best(population): 
    best_indivual = {}
    for individ in population:
        if (_validate_individ(individ)):
            best_indivual = individ
            return best_indivual

In [28]:
population = _starting_population() #стартовая популяция
counter = 0
for i in range(0, GENERATIONS):
    old_population, selected_population = selection(population) #популяция проходит отбор (20% самых приспособленных)
    child_population, not_used_parents = _get_children(selected_population) #скрещивание, так же возвращаются родители без пары
    old_population += not_used_parents
    mutated_population = _mutation(child_population) #мутация
    new_population = selected_population + selected_population + selected_population + selected_population +  mutated_population

    population = sorted(new_population, key = lambda i: i['value'], reverse = True)
    counter += 1
    if(_check_population(population) < 10):
        break;


print('counter', counter)
kek,kok = selection(population)
best_indivual = _find_best(population)
print(best_indivual)

counter 13
{'gen': '111011111011111100000100000100', 'weight': 12978.0, 'volume': 11.9, 'value': 4404.0}


In [39]:
selected = []
best_genome = list(map(int,list(best_indivual['gen'])))
for index, g in enumerate(best_genome):
    if g == 1:
        selected.append({'weight': data[index][0], 'volume': data[index][1], 'value': data[index][2]})
    
total_value = sum([item['value'] for item in selected])
total_weight = sum([item['weight'] for item in selected])
total_volume = sum([item['volume'] for item in selected])

print("Total value: ", total_value)
print("Total weight: ", total_weight)
print("Total volume: ",round(total_volume,2))
print()
for i in selected:
    print(i)

Total value:  4404.0
Total weight:  12978.0
Total volume:  11.9

{'weight': 1149.0, 'volume': 1.2, 'value': 384.0}
{'weight': 602.0, 'volume': 1.1, 'value': 374.0}
{'weight': 1278.0, 'volume': 0.5, 'value': 351.0}
{'weight': 1474.0, 'volume': 0.7, 'value': 321.0}
{'weight': 740.0, 'volume': 1.0, 'value': 317.0}
{'weight': 154.0, 'volume': 0.7, 'value': 308.0}
{'weight': 1446.0, 'volume': 0.5, 'value': 297.0}
{'weight': 174.0, 'volume': 0.9, 'value': 297.0}
{'weight': 893.0, 'volume': 0.9, 'value': 280.0}
{'weight': 864.0, 'volume': 0.4, 'value': 244.0}
{'weight': 856.0, 'volume': 0.6, 'value': 238.0}
{'weight': 945.0, 'volume': 0.8, 'value': 236.0}
{'weight': 133.0, 'volume': 0.5, 'value': 234.0}
{'weight': 579.0, 'volume': 0.4, 'value': 229.0}
{'weight': 1112.0, 'volume': 1.1, 'value': 173.0}
{'weight': 579.0, 'volume': 0.6, 'value': 121.0}
